Import des data Pays dans T_Pays depuis la table T_complete
A partir d'une requete SELECT DISTINCT sur Pays


In [ ]:
import psycopg2
import os
import csv
import pandas as pd
#import re

def chemin_fichier()-> str :
    """
        Donne le nom du chemin complet du fichier actuel (.py) 
    """
    chemin=os.getcwd() #methode pour tourner avec jupyter
    #chemin= path.dirname(path.abspath(__file__)) #methode d'Alice avec <os>
    #chemin = str(Path(__file__).parent) #ma methode avec <path>
    #print(chemin)
    return chemin

# etablir une connection
def connexion_DB(fichierDB):
    try:
        conn = psycopg2.connect(user = "santa",
                                password = "claus",
                                host = "localhost",
                                port = "5432:5432",
                                database = fichierDB
                                )

        cur = conn.cursor()
        # Print PostgreSQL Connection properties
        #print ( "Proprietes : ",conn.get_dsn_parameters(),"\n")

        # Print PostgreSQL version
        cur.execute("SELECT version();")
        record = cur.fetchone()
        #print("Version de PostgreSQL : ", record,"\n")
        cur.close()
    except (Exception, psycopg2.Error) as error :
        print ("Error while connecting to PostgreSQL", error)
    
    return conn

def deconnexion_DB(conn):
#closing database connection.
    if(conn):
        conn.close()
        print("PostgreSQL connection is closed")

conn=connexion_DB("santa_data")

path_dossier_letters=chemin_fichier() +'/Partie_1/lettres_pere_noel'

cursor = conn.cursor()


In [ ]:
# analyse des differentes occurences du champ 'Pays'

rq_select='''
          SELECT
          count(ID),
          Pays
          FROM T_complete
          WHERE Pays IS NOT NULL
          GROUP BY Pays
          ORDER BY Pays;
          '''    
cursor.execute(rq_select)
res=cursor.fetchall()

print('\nNb'.ljust(8),'pays'.ljust(15))
trait="-"*40
print(trait)
for row in res:
    print(str(row[0]).ljust(8),end='')
    print(str(row[1]).ljust(15))




In [ ]:
#insertion de données Pays dans T_Pays
try:
    conn=connexion_DB("santa_data")
    cursor = conn.cursor()
    
    #suppression des enregistrements de la T_pays avant de refaire tourner le programme
    cursor.execute("DELETE FROM T_pays")
  
    count=0
    
    rq_select='''
          SELECT DISTINCT
          Pays
          FROM T_complete
          WHERE pays IS NOT NULL
          ORDER BY Pays;
          '''    
    cursor.execute(rq_select)
    l_pays=cursor.fetchall()
    for elt_pays in l_pays:
        insert_data_query = f""" 
                        INSERT INTO T_pays
                        (ID_p,Pays) 
                        VALUES (default,'{elt_pays[0]}')
                        """
        cursor.execute(insert_data_query)
        #nb_l+=1
        conn.commit()
        count += cursor.rowcount
    print (count, "Record inserted successfully into T_Pays")
    
    cursor.execute("SELECT * FROM T_pays")
    #resultats=cur.execute(select_SQL)
    res=cursor.fetchall()
    print('\nID_p'.ljust(8),'pays'.ljust(20))
    trait="-"*40
    print(trait)
    for row in res:
        print(str(row[0]).ljust(8),end='')
        print(str(row[1]).ljust(15))

    
except (Exception, psycopg2.Error) as error :
    if(conn):
        print("Failed to insert record into table T_Pays", error)

finally:
    #closing database connection.
    if(conn):
        cursor.close()
        deconnexion_DB(conn)